In [6]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [7]:
!pip install pygame

In [8]:
import sys
import pygame
import time
import random
from pygame.locals import *

In [38]:
#初始化

pygame.init()
fpsClock = pygame.time.Clock() ###
playSurface = pygame.display.set_mode((640, 480))
pygame.display.set_caption('Snake Go!')

#顏色變量
red = pygame.Color(255, 0, 0)
black = pygame.Color(0, 0, 0)
white = pygame.Color(255, 255, 255)
yellow = pygame.Color(255, 224, 32)
grey = pygame.Color(150, 150, 150)
lightred = pygame.Color(225, 208, 160)
lightgrey = pygame.Color(220, 220, 220)

fps_controller = pygame.time.Clock()

#貪食蛇 & 樹莓

snakePosition = [100, 100] #頭
snakeSegments = [[100, 100], [80, 100], [60, 100]] #身體
##定義正確框框位置與錯誤框框位置
x = np.random.randint(1, 32)
y = np.random.randint(1, 24)
X = int(x*20)
Y = int(y*20)
raspberryPosition = [X, Y] #初始樹莓位置
x1 = np.random.randint(1, 32)
y1 = np.random.randint(1, 24)
while x1==x:
    x1=np.random.randint(1, 32)
while y1==y:
    y1=np.random.randint(1, 24)
X1 = int(x1*20)
Y1 = int(y1*20)
raspberryPosition1 = [X1, Y1]
x2 = np.random.randint(1, 32)
y2 = np.random.randint(1, 24)
while x2==x or x2==x1:
    x2=np.random.randint(1, 32)
while y2==y or y2==y1:
    y1=np.random.randint(1, 24)
X2 = int(x2*20)
Y2 = int(y2*20)
raspberryPosition2 = [X2, Y2]

##定義錯誤答案c1c2與正確答案c
b = np.random.randint(1, 10)
a = np.random.randint(1, 10)
c = a+b
c1=np.random.randint(0, 19)
while c1==c:
    c1=np.random.randint(0, 19)
c2=np.random.randint(0, 19)
while c2==c or c2==c1:
    c2=np.random.randint(0, 19)
    
food_spawn = True

direction = 'RIGHT' #初始方向
score = 0 #初始分數

changeDirection = direction

##設計題目
def question():
    questionFont = pygame.font.SysFont('times new roman', 40)
    questionSurf = questionFont.render('Question: '+ str(a) + '+' + str(b) + '= ?', True, lightred)
    questionRect = questionSurf.get_rect()
    questionRect.midtop = (320, 80)
    playSurface.blit(questionSurf, questionRect)
def answer():
    realanswerFont = pygame.font.SysFont(str(c),40)
    realanswerSurf = realanswerFont.render(str(c) , True, yellow)
    realanswerRect = realanswerSurf.get_rect()
    realanswerRect.midtop = (X, Y)
    playSurface.blit(realanswerSurf, realanswerRect)
def answer1():
    realanswerFont = pygame.font.SysFont(str(c1),40)
    realanswerSurf = realanswerFont.render(str(c1) , True, yellow)
    realanswerRect = realanswerSurf.get_rect()
    realanswerRect.midtop = (X1, Y1)
    playSurface.blit(realanswerSurf, realanswerRect)
def answer2():
    realanswerFont = pygame.font.SysFont(str(c2),40)
    realanswerSurf = realanswerFont.render(str(c2) , True, yellow)
    realanswerRect = realanswerSurf.get_rect()
    realanswerRect.midtop = (X2, Y2)
    playSurface.blit(realanswerSurf, realanswerRect)

#Gameover & Score
def gameover():
    # 字體大小
    gameoverFont = pygame.font.SysFont('times new roman', 72)
    gameoverSurf = gameoverFont.render('Game Over', True, grey)
    gameoverRect = gameoverSurf.get_rect()
    gameoverRect.midtop = (320, 125)   ###
    playSurface.blit(gameoverSurf, gameoverRect)###
    show_score(0, red, 'times', 20)
    pygame.display.flip()  ###刷新介面
    #休眠10秒自動關閉
    time.sleep(10)
    pygame.quit()
    sys.exit()
    #分數與字體大小
def show_score(choice, color, font, size):
    scoreFont = pygame.font.SysFont('times new roman', 20)
    scoreSurf = scoreFont.render('Score: ' + str(score), True, color)
    scoreRect = scoreSurf.get_rect()
    scoreRect.midtop = (320, 225)
    playSurface.blit(scoreSurf, scoreRect) ###    
    
    
#鍵盤
#檢測按鍵
while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        elif event.type == pygame.KEYDOWN:
            #判斷鍵盤
            if event.key == pygame.K_RIGHT or event.key == ord('d'):
                changeDirection = 'RIGHT'
            if event.key == pygame.K_LEFT or event.key == ord('a'):
                changeDirection = 'LEFT'
            if event.key == pygame.K_UP or event.key == ord('w'):
                changeDirection = 'UP'
            if event.key == pygame.K_DOWN or event.key == ord('s'):
                changeDirection = "DOWN"
            if event.key == pygame.K_ESCAPE: #按esc退出遊戲
                pygame.event.post(pygame.event.Event(pygame.QUIT))
        #限制反向運動
    if changeDirection == 'RIGHT' and not direction == 'LEFT':
         direction = changeDirection
    if changeDirection == 'LEFT' and not direction == 'RIGHT':
         direction = changeDirection
    if changeDirection == 'UP' and not direction == 'DOWN':
        direction = changeDirection
    if changeDirection == 'DOWN' and not direction == 'UP':
        direction = changeDirection
        #根據方向移動頭座標
    if direction == 'RIGHT':
        snakePosition[0] += 20
    if direction == 'LEFT':
        snakePosition[0] -= 20
    if direction == 'UP':
        snakePosition[1] -= 20 #####
    if direction == 'DOWN':
        snakePosition[1] += 20
        
        #將頭位置加入列表
    snakeSegments.insert(0, list(snakePosition))

#判斷是否吃到樹莓
    if snakePosition[0] == raspberryPosition[0] and snakePosition[1] == raspberryPosition[1]:
        score += 1 #分數增加
        food_spawn = False
    else:
        snakeSegments.pop()  #將蛇身最後移出列表

#重新生成樹莓
    if not food_spawn:
        b = np.random.randint(1, 10)
        a = np.random.randint(1, 10)
        c = a+b
        c1=np.random.randint(0, 19)
        while c1==c:
            c1=np.random.randint(0, 19)
        c2=np.random.randint(0, 19)
        while c2==c or c2==c1:
            c2=np.random.randint(0, 19)
        x = np.random.randint(1, 32)
        y = np.random.randint(1, 24)
        X = int(x*20)
        Y = int(y*20)
        raspberryPosition = [X, Y]
        x1 = np.random.randint(1, 32)
        y1 = np.random.randint(1, 24)
        while x1==x:
            x1=np.random.randint(1, 32)
        while y1==y:
            y1=np.random.randint(1, 24)
        X1 = int(x1*20)
        Y1 = int(y1*20)
        raspberryPosition1 = [X1, Y1]
        x2 = np.random.randint(1, 32)
        y2 = np.random.randint(1, 24)
        while x2==x or x2==x1:
            x2=np.random.randint(1, 32)
        while y2==y or y2==y1:
            y1=np.random.randint(1, 24)
        X2 = int(x2*20)
        Y2 = int(y2*20)
        raspberryPosition2 = [X2, Y2]
        food_spawn = True
        
#繪製pygame的顯示層
    playSurface.fill(black)
    for position in snakeSegments[1:]: #蛇身白色
        pygame.draw.rect(playSurface, white, Rect(position[0], position[1], 20, 20))
    pygame.draw.rect(playSurface, lightgrey, Rect(snakePosition[0], snakePosition[1], 20, 20)) #頭灰
    pygame.draw.rect(playSurface, red, Rect(raspberryPosition[0], raspberryPosition[1], 20, 20))
    pygame.draw.rect(playSurface, red, Rect(raspberryPosition1[0], raspberryPosition1[1], 20, 20))
    pygame.draw.rect(playSurface, red, Rect(raspberryPosition2[0], raspberryPosition2[1], 20, 20))#樹莓紅
#刷新pygame顯示層
    pygame.display.flip()

#判斷死亡
    if snakePosition[0] > 620 or snakePosition[0] < 0:
        gameover()
    if snakePosition[1] > 460 or snakePosition[1] < 0:
        gameover()
    for snakeBody in snakeSegments[1:]: #碰身體
        if snakePosition[0] == snakeBody[0] and snakePosition[1] == snakeBody[1]:
            gameover()
    for snakeBody in snakeSegments[1:]: #碰錯誤答案
        if snakePosition[0] == raspberryPosition1[0] and snakePosition[1] == raspberryPosition1[1]:
            gameover()
    for snakeBody in snakeSegments[1:]: #碰錯誤答案
        if snakePosition[0] == raspberryPosition2[0] and snakePosition[1] == raspberryPosition2[1]:
            gameover()
    

# 遊戲畫面顯示   
    show_score(1, white, score, 20)
    question()
    answer()
    answer1()
    answer2()
#控制遊戲速度
    if len(snakeSegments) <40:
        speed = 3 + len(snakeSegments)//2
    else:
        speed = 23
    fpsClock.tick(500)
    pygame.display.update()
    fps_controller.tick(speed)

SystemExit: 